<a href="https://colab.research.google.com/github/jpastor1649/Models-And-Simulation/blob/main/Simulations/Ejercicio_modelos_2_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

.  Una cadena hotelera tiene dos buses para recoger y dejar personas en un aeropuerto local y dos hoteles separados. Los buses viajan desde el aeropuerto al hotel 1, luego al hotel 2, y regresan al aeropuerto para continuar con este patrón. El tiempo de viaje entre cada lugar sigue una distribución normal con una media de 20 y una desviación estándar de 2 minutos. El tiempo de llegada de los viajeros desde sus vuelos se distribuye exponencial con una media de 2.5 minutos. Cincuenta por ciento de las personan se bajan en el primer hotel, y el bus recoge personas de este hotel que desean ir al aeropuerto. El otro cincuenta por ciento de las personas se baja en el segundo hotel, y el bus recoge nuevamente personas. En el aeropuerto, todo el mundo se baja. En ambos hoteles las personas llegan al paradero del bus para ir al aeropuerto con tiempos entre llegadas exponenciales con media de 5 minutos. Simular el sistema donde el primer bus sale del aeropuerto al iniciar la simulación y el segundo sale del aeropuerto 30 minutos después del primero. Determine la cantidad de asientos requeridos en ambos buses tal que cualquier persona esperando pueda ser recogida.

In [ ]:
pip install simpy

In [ ]:
"""
Simulation of a transportation system between the airport and hotels
--------------------------------------------------------------
Discrete simulation model using SimPy.

Autor: [jpastor]
Fecha: 2025-05-24
Licencia: MIT
Descripción:
    This script simulates a system of buses that transport passengers between
    an airport and two hotels. The objective is to determine the minimum capacity
    per bus that guarantees that no passengers are waiting without being transported.

TODO:
    - Add detaiiled performance metrics
    - Parameterize system configuration
    - Use deque instead of list for queuing efficiency
    - Encapsulate bus logic in a class
    - Properly comment out each logic block
    - Add docstrings to key functions
    - Parameterize buses for scalability
    - Restructure logic to make the code more scalable

"""
import simpy
import random
from collections import deque

In [ ]:
SIM_TIME = 60 * 24
MEAN_ARRIVAL_TIME_AIRPORT = 2.5
MEAN_ARRIVAL_TIME_HOTEL = 5
MEAN_BUS_TIME_STATIONS = 20
STD_BUS_TIME = 2
BUS_START_DELAYS = [0,30]

def arrival_airport_p(env, airport_to_hotel1, airport_to_hotel2):
    toggle = True
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_AIRPORT))
        if toggle:
            airport_to_hotel1.append(1)
        else:
            airport_to_hotel2.append(1)
        toggle = not toggle

def arrival_hotel_p(env, cola_h):
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_HOTEL))
        cola_h.append(1)

def board_passengers(queue, max_board):
    boarded = 0
    while queue and boarded < max_board:
        queue.popleft()
        boarded += 1
    return boarded

def update_load(boarded, onboard, occupancy, space):
    onboard += boarded
    occupancy += boarded
    space -= boarded
    return onboard, occupancy, space    #onboard,occupancy,space updated

def unload_passengers(onboard, occupancy):
    occupancy = max(occupancy - onboard, 0)
    return 0, occupancy  # onboard reset, occupancy updated

def bus_trips(env, id,start_delay, capacity, airport_to_hotel1, airport_to_hotel2, hotel1_to_airport, hotel2_to_airport, failures):
    yield env.timeout(start_delay)

    onboard_H1 = onboard_H2 = 0
    onboard_A = 0
    occupancy = 0

    while True:
        # 1. Pick up passengers in airport
        space = capacity
        total_air_h = len(airport_to_hotel1) + len(airport_to_hotel2)

        if total_air_h > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Airport")

        to_board_min_H1 = min(space // 2, len(airport_to_hotel1))
        boarded_H1 = board_passengers(airport_to_hotel1, to_board_min_H1)
        onboard_H1, occupancy, space = update_load(boarded_H1, onboard_H1, occupancy, space)

        to_board_min_H2 = min(space, len(airport_to_hotel2))
        boarded_H2 = board_passengers(airport_to_hotel2, to_board_min_H2)
        onboard_H2, occupancy, space = update_load(boarded_H2, onboard_H2, occupancy, space)


        # 2. Airport → Hotel 1
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_H1, occupancy = unload_passengers(onboard_H1, occupancy)
        space = capacity - occupancy


        if len(hotel1_to_airport) > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Hotel 1")
        boarded_from_H1 = board_passengers(hotel1_to_airport, space)
        onboard_A, occupancy, space = update_load(boarded_from_H1, onboard_A, occupancy, space)


        # 3. Hotel 1 → Hotel 2
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_H2, occupancy = unload_passengers(onboard_H2, occupancy)
        space = capacity - occupancy


        if len(hotel2_to_airport) > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Hotel 2")
        boarded_from_H2 = board_passengers(hotel2_to_airport, space)
        onboard_A, occupancy, space = update_load(boarded_from_H2, onboard_A, occupancy, space)


        # 4. Hotel 2 → Airport
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_A,occupancy = unload_passengers(onboard_A,occupancy)

def simulation(capacity):
        env = simpy.Environment()
        airport_to_hotel1, airport_to_hotel2 = deque(), deque()
        hotel1_to_airport, hotel2_to_airport = deque(), deque()
        failures = []

        env.process(arrival_airport_p(env, airport_to_hotel1, airport_to_hotel2))
        env.process(arrival_hotel_p(env, hotel1_to_airport))
        env.process(arrival_hotel_p(env, hotel2_to_airport))
        env.process(bus_trips(env, 'Bus1', BUS_START_DELAYS[0],capacity, airport_to_hotel1, airport_to_hotel2, hotel1_to_airport, hotel2_to_airport, failures))
        env.process(bus_trips(env, 'Bus2', BUS_START_DELAYS[1],capacity, airport_to_hotel1, airport_to_hotel2, hotel1_to_airport, hotel2_to_airport, failures))

        env.run(until=SIM_TIME)
        return failures

def minimum_capacity(rep=1000, max_cap=100):
    for capacity in range(10, max_cap + 1):
        sim_success = 0
        for _ in range(rep):
            failures = simulation(capacity)
            if not failures:
                sim_success += 1
        print(f"Capacity for {capacity} passengers: {sim_success / rep * 100:.2f}% success rate")
        if sim_success == rep:
            return capacity
    return None


print("Minimum capacity per bus:", minimum_capacity())

Capacity for 10 passengers: 0.00% success rate
Capacity for 11 passengers: 0.00% success rate
Capacity for 12 passengers: 0.00% success rate
Capacity for 13 passengers: 0.00% success rate
Capacity for 14 passengers: 0.00% success rate
Capacity for 15 passengers: 0.00% success rate
Capacity for 16 passengers: 0.00% success rate
Capacity for 17 passengers: 0.00% success rate
Capacity for 18 passengers: 0.00% success rate
Capacity for 19 passengers: 0.10% success rate
Capacity for 20 passengers: 1.00% success rate
Capacity for 21 passengers: 2.40% success rate
Capacity for 22 passengers: 5.30% success rate
Capacity for 23 passengers: 12.50% success rate
Capacity for 24 passengers: 18.30% success rate
Capacity for 25 passengers: 25.50% success rate
Capacity for 26 passengers: 37.10% success rate
Capacity for 27 passengers: 46.10% success rate
Capacity for 28 passengers: 53.50% success rate
Capacity for 29 passengers: 60.50% success rate
Capacity for 30 passengers: 68.80% success rate
Capac

In [ ]:
#import simpy
import random

# Configuraciones
SIM_TIME = 60 * 24  # Minutos en un día
MEAN_ARRIVAL_TIME_AIRPORT = 2.5
MEAN_ARRIVAL_TIME_HOTEL = 5
MEAN_BUS_TIME_STATIONS = 20
STD_BUS_TIME = 2

class Passenger:
    def __init__(self, origin, destination, arrival_time):
        self.origin = origin
        self.destination = destination
        self.arrival_time = arrival_time


def llegada_pasajeros_aeropuerto(env, store_H1, store_H2):
    toggle = True
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_AIRPORT))
        dest = 'H1' if toggle else 'H2'
        passenger = Passenger('A', dest, env.now)
        if toggle:
            yield store_H1.put(passenger)
        else:
            yield store_H2.put(passenger)
        toggle = not toggle


def llegada_pasajeros_hotel(env, queue, hotel_name):
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_HOTEL))
        passenger = Passenger(hotel_name, 'A', env.now)
        yield queue.put(passenger)


def bus(env, name, start_delay, capacity, air_H1, air_H2, queue_H1, queue_H2, log):
    yield env.timeout(start_delay)

    while True:
        onboard = []
        space = capacity

        # Cargar en aeropuerto balanceado
        while space > 0 and (len(air_H1.items) > 0 or len(air_H2.items) > 0):
            if len(air_H1.items) >= len(air_H2.items):
                if len(air_H1.items) > 0:
                    p = yield air_H1.get()
                    onboard.append(p)
                    space -= 1
                elif len(air_H2.items) > 0:
                    p = yield air_H2.get()
                    onboard.append(p)
                    space -= 1
            else:
                if len(air_H2.items) > 0:
                    p = yield air_H2.get()
                    onboard.append(p)
                    space -= 1
                elif len(air_H1.items) > 0:
                    p = yield air_H1.get()
                    onboard.append(p)
                    space -= 1

        if len(air_H1.items) + len(air_H2.items) > 0:
            log.append(f"{env.now:.2f} - {name} dejó gente esperando en aeropuerto")

        # Viajar a Hotel 1
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard = [p for p in onboard if p.destination != 'H1']
        space = capacity - len(onboard)

        # Subir pasajeros de Hotel 1
        while space > 0 and len(queue_H1.items) > 0:
            p = yield queue_H1.get()
            onboard.append(p)
            space -= 1
        if len(queue_H1.items) > 0:
            log.append(f"{env.now:.2f} - {name} dejó gente esperando en Hotel 1")

        # Viajar a Hotel 2
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard = [p for p in onboard if p.destination != 'H2']
        space = capacity - len(onboard)

        # Subir pasajeros de Hotel 2
        while space > 0 and len(queue_H2.items) > 0:
            p = yield queue_H2.get()
            onboard.append(p)
            space -= 1
        if len(queue_H2.items) > 0:
            log.append(f"{env.now:.2f} - {name} dejó gente esperando en Hotel 2")

        # Viajar al aeropuerto
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard = []  # Se bajan todos los pasajeros que iban al aeropuerto


def simular(capacidad):
    env = simpy.Environment()
    air_H1 = simpy.Store(env)
    air_H2 = simpy.Store(env)
    queue_H1 = simpy.Store(env)
    queue_H2 = simpy.Store(env)
    log = []

    env.process(llegada_pasajeros_aeropuerto(env, air_H1, air_H2))
    env.process(llegada_pasajeros_hotel(env, queue_H1, 'H1'))
    env.process(llegada_pasajeros_hotel(env, queue_H2, 'H2'))
    env.process(bus(env, 'Bus1', 0, capacidad, air_H1, air_H2, queue_H1, queue_H2, log))
    env.process(bus(env, 'Bus2', 30, capacidad, air_H1, air_H2, queue_H1, queue_H2, log))

    env.run(until=SIM_TIME)
    return log


def encontrar_capacidad_minima():
    rep = 500
    for cap in range(10, 50):
        exitos = 0
        for _ in range(rep):
            log = simular(cap)
            if not log:
                exitos += 1
        print(f"Capacidad {cap}: {exitos / rep * 100:.2f}% de éxito")
        if exitos == rep:
            return cap
    return None

if __name__ == '__main__':
    random.seed()
    print("Capacidad mínima por bus:", encontrar_capacidad_minima())


Capacidad 10: 0.00% de éxito
Capacidad 11: 0.00% de éxito
Capacidad 12: 0.00% de éxito
Capacidad 13: 0.00% de éxito
Capacidad 14: 0.00% de éxito
Capacidad 15: 0.00% de éxito
Capacidad 16: 0.00% de éxito
Capacidad 17: 0.00% de éxito
Capacidad 18: 0.00% de éxito
Capacidad 19: 0.20% de éxito
Capacidad 20: 0.60% de éxito
Capacidad 21: 2.40% de éxito
Capacidad 22: 5.60% de éxito
Capacidad 23: 12.60% de éxito
Capacidad 24: 17.60% de éxito
Capacidad 25: 27.00% de éxito
Capacidad 26: 36.00% de éxito
Capacidad 27: 46.60% de éxito
Capacidad 28: 50.80% de éxito
Capacidad 29: 62.60% de éxito
Capacidad 30: 75.20% de éxito
Capacidad 31: 75.60% de éxito
Capacidad 32: 85.40% de éxito
Capacidad 33: 88.80% de éxito
Capacidad 34: 93.80% de éxito
Capacidad 35: 96.60% de éxito
Capacidad 36: 97.40% de éxito
Capacidad 37: 99.40% de éxito
Capacidad 38: 99.20% de éxito
Capacidad 39: 99.20% de éxito
Capacidad 40: 99.80% de éxito
Capacidad 41: 99.80% de éxito
Capacidad 42: 100.00% de éxito
Capacidad mínima por b